#Intro
This notebook was create with the taught to train yolov5 on zoomed in images of COTS but this idea was later abandomed as it was unlikely to improve the kaggle score.


# Import libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

# Data Processing
See ak-dataprocessing

In [ ]:
TRAIN_PATH = "../input/tensorflow-great-barrier-reef/train_images"
HEIGHT, WIDTH = 720, 1280
image_size = 640

vid0_path = "../input/tensorflow-great-barrier-reef/train_images/video_0"
vid1_path = "../input/tensorflow-great-barrier-reef/train_images/video_1"
vid2_path = "../input/tensorflow-great-barrier-reef/train_images/video_2"
vid_paths = [vid0_path, vid1_path, vid2_path]

vid0_ls = [os.path.join(vid0_path,f) for f in os.listdir(vid0_path)]
vid0_ls = sorted(vid0_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

vid1_ls = [os.path.join(vid1_path,f) for f in os.listdir(vid1_path)]
vid1_ls = sorted(vid1_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

vid2_ls = [os.path.join(vid2_path,f) for f in os.listdir(vid2_path)]
vid2_ls = sorted(vid2_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))
files_ls = [vid0_ls, vid1_ls, vid2_ls]

train_df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv",
                       sep = r',', skipinitialspace = True)

In [ ]:
print(vid0_ls[:3])
print("\n",files_ls[0][:3])

In [ ]:
print(len(files_ls[0])+len(files_ls[1])+len(files_ls[2]))
print(len(train_df))

In [ ]:
train_df.head(5)

#### Helper functions

In [ ]:
def get_oldpath(x):
    if x.video_id == 0:
        path = os.path.join(vid0_path,str(x.video_frame)+".jpg")
    elif x.video_id == 1:
        path = os.path.join(vid1_path,str(x.video_frame)+".jpg")
    else:
        path = os.path.join(vid2_path,str(x.video_frame)+".jpg")
        
    return path

# def get_newpath(x):
#     filename = f"{x.video_id}_{x.video_frame}.jpg"
#     return os.path.join("./dataset", filename)

def get_filename(x):
    return f"{x.video_id}_{x.video_frame}.jpg"

In [ ]:
import ast
from tqdm import tqdm
import shutil

#train_df = train_df[train_df.annotations != "[]"]
train_df["annotations"] = train_df["annotations"].map(lambda x : ast.literal_eval(x))

train_df["filepath"] = train_df.apply(lambda x : get_oldpath(x), axis=1)
# train_df["newpath"] = train_df.apply(lambda x : get_newpath(x), axis=1)
train_df["filename"] = train_df.apply(lambda x : get_filename(x), axis=1)

# os.makedirs("./dataset",exist_ok=True)
# for i in tqdm(range(len(train_df))):
#     src = train_df.iloc[i]["filepath"]
#     dst = train_df.iloc[i]["newpath"]
#     if os.path.exists(dst)==False:
#         shutil.copy(src,dst)
    
train_df.sample(5, random_state=24)

#### Main Dataframe

In [ ]:
df = train_df[train_df["annotations"].str.len() != 0].copy()
df = df.explode("annotations")

df["width"] = df.apply(lambda x : x.annotations["width"], axis=1)
df["height"] = df.apply(lambda x : x.annotations["height"], axis=1)
# df["label"] = ["starfish"]*len(df)

df["xmin"] = df.apply(lambda x : x.annotations["x"], axis=1)
df["ymin"] = df.apply(lambda x : x.annotations["y"], axis=1)
df["xmax"] = df.apply(lambda x : x.annotations["x"]+x.annotations["width"], axis=1)
df["ymax"] = df.apply(lambda x : x.annotations["y"]+x.annotations["height"], axis=1)

df.loc[df["xmax"] > WIDTH, "xmax"] = WIDTH
df.loc[df["ymax"] > HEIGHT, "ymax"] = HEIGHT

df["yolo_bbox"] = df.apply(lambda x : [int((x.xmax+x.xmin)/2)/WIDTH , int((x.ymax+x.ymin)/2)/HEIGHT ,x.width/WIDTH, x.height/HEIGHT ], axis=1)

df = df.drop(["sequence","video_frame","video_id","sequence_frame","image_id","annotations"], axis=1)
df = df.reset_index(drop=True)
df.sample(5, random_state=24)

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as npimg
import cv2

In [ ]:
def draw_annotations(spot_fish_img, img_row):
    x = df[df['filename']==img_row["filename"]].copy()
    print(f"Number of starfish found is {len(x)}")
    for index in x.index:
        spot_fish_img = cv2.rectangle(spot_fish_img, (x['xmin'][index], x['ymin'][index]),
                  (x['xmax'][index], x['ymax'][index]), (255,255,0), 2)
    return spot_fish_img

#Plot a random image with starfish shown

In [ ]:
img_row = df.iloc[random.randint(0, len(df))]

original_img = npimg.imread(img_row['filepath'])

fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()

axs[0].imshow(original_img)
axs[0].set_title('Original Image')

spot_fish_img = original_img.copy()
draw_annotations(spot_fish_img, img_row)

axs[1].imshow(spot_fish_img)
axs[1].set_title('spotfish Image')

# Determine the mean starfish size
Cropping height and width is chosen to be the mean starfish size plus one standard deviation. 

In [ ]:
width_df = df["xmax"]-df["xmin"]
print("max width =",width_df.max()," min width =", width_df.min()," mean width =",width_df.mean(),"  std width =",width_df.std() )
print("<x>+sigma=", width_df.mean()+width_df.std())
crop_width = width_df.mean()+width_df.std()#width_df.max()
crop_width = int(crop_width)
if crop_width%2 !=0:
    crop_width = crop_width+1
if crop_width> WIDTH:
    print("crop_width> WIDTH")


height_df = df["ymax"]-df["ymin"]
print("max height=", height_df.max()," min height=",height_df.min()," mean height=",height_df.mean()," std height=", height_df.std())
print("<x>+sigma=", height_df.mean()+height_df.std())
crop_height = height_df.mean()+height_df.std()#height_df.max()
crop_height = int(crop_height)
if crop_height%2 !=0:
    crop_height = crop_height+1
if crop_height> HEIGHT:
    print("crop_height> HEIGHT")
    
print("crop height",crop_height,"crop width", crop_width)

# Function to determine the parameters needed to crop image arround ech starfish

In [ ]:
def get_crop_box(center):
    #center = [x_center, y_center]
    #crop_box = [y_min ,y_max, x_min, x_max]
    crop_box = [int(center[1]-crop_height/2),int(center[1]+crop_height/2),
                int(center[0]-crop_width/2),int(center[0]+crop_width/2)]
    #shift = [x_shift, y_shift]
    shift = [0,0]
    #If box to be cropped is out of bounds of the image shift up/down/left/right
    #print("old cropbox ",crop_box )
    if crop_box[0] < 0:
        crop_box[1] = crop_box[1]-crop_box[0]
        shift[1] = -crop_box[1]
        crop_box[0] = 0
        #print("new cropbox ",crop_box )
    if crop_box[1] > HEIGHT:
        crop_box[0] = crop_box[0]-(crop_box[1]-HEIGHT)
        shift[1] = -(crop_box[1]-HEIGHT)
        crop_box[1] = HEIGHT
        #print("new cropbox ",crop_box )
    if crop_box[2] < 0:
        crop_box[3] = crop_box[3]-crop_box[2]
        shift[0] = -crop_box[2]
        crop_box[2] = 0
        #print("new cropbox ",crop_box )
    if crop_box[3] > WIDTH:
        crop_box[2] = crop_box[2]-(crop_box[3]-WIDTH)
        shift[0] = -(crop_box[3]-WIDTH)
        crop_box[3] = WIDTH
        #print("new cropbox ",crop_box )
    
#     if (crop_box[1]-crop_box[0]) !=crop_height:
#         print('height crop error',(crop_box[1]-crop_box[0]),crop_height)
#     if (crop_box[3]-crop_box[2]) !=crop_width:
#         print('width crop error',(crop_box[3]-crop_box[2]),crop_width)
    return crop_box, shift

#Test cropping pipeline and system to determine bounding box in new cropped image

In [ ]:
x = df[df['filename']==img_row["filename"]].copy()
print(f"Number of starfish found is {len(x)}")

for index in x.index:
    ###########################
    #tight crop
#     img_crop = original_img[x['ymin'][index]:x["ymax"][index],x['xmin'][index]:x["xmax"][index]]
    
    ###########################
    #equal sized crops
    center = [np.ceil((x["xmax"][index]+x['xmin'][index])/2) , np.ceil((x["ymax"][index]+x['ymin'][index])/2)]
    crop_box, shift = get_crop_box(center)
    img_crop = npimg.imread(img_row['filepath'])
    img_crop = img_crop[crop_box[0]:crop_box[1],crop_box[2]:crop_box[3]]
    ##########################
    f1 = plt.figure()
    f1.clf()
#     img_crop = cv2.rectangle(img_crop, (x['xmin'][index], x['ymin'][index]),
#                   (x['xmax'][index], x['ymax'][index]), (255,255,0), 2)
    print(shift)
    
    new_bbox = [shift[0]/crop_width, shift[1]/crop_height, x['width'][index]/crop_width, x['height'][index]/crop_height]
    if new_bbox[2] > 1 or new_bbox[3]>1:
        print(new_bbox)
    #print(new_bbox)
    
    img_crop = cv2.rectangle(img_crop, (int((crop_width-x['width'][index])/2)-shift[0], int((crop_height-x['height'][index])/2-shift[1])),
              (int((crop_width+x['width'][index])/2)-shift[0], int((crop_height+x['height'][index])/2)-shift[1]), (255,255,0), 1)
    plt.imshow(img_crop)


# Crop images and save to folder. Add new bounding box annontations to dataframe

In [ ]:
!mkdir -p './images'

df["new_bbox"] = ''
for index in tqdm(df.index):#range(4780,4786,1): # 
    #crop images that contain starfish
    filename = f"./images/{index}_{df['filename'][index]}"
    
    center = [np.ceil((df["xmax"][index]+df['xmin'][index])/2) , np.ceil((df["ymax"][index]+df['ymin'][index])/2)]
    crop_box, shift = get_crop_box(center)
    
    new_bbox = [shift[0]/crop_width, shift[1]/crop_height, df['width'][index]/crop_width, df['height'][index]/crop_height]
    df["new_bbox"][index] = new_bbox
    
    if (os.path.exists(filename)==False): # or (os.path.exists(filename)==True):#
        original_img = cv2.imread(df["filepath"][index]) 
        #######
        #equal sized crops
        img_crop = original_img[crop_box[0]:crop_box[1],crop_box[2]:crop_box[3]] 
        #######
        #tight crop
#         img_crop = original_img[df_training_clas['ymin'][index]:df_training_clas["ymax"][index],df_training_clas['xmin'][index]:df_training_clas["xmax"][index]]
        #######
        cv2.imwrite(filename,img_crop)
        
    

In [ ]:
!mkdir -p './labels'

for index in tqdm(df.index):
    
    row_data = df.iloc[index,:]
    height = row_data["height"]
    width = row_data["width"]
    
    label_filename = row_data["filename"].split(".")
    label_filename = label_filename[0]
    
    #Create file and write in it
    with open(f"./labels/{index}_{label_filename}.txt", 'w') as file:
        # Convert coco format to yolo format
        yolo_bbox = row_data["new_bbox"]
        if yolo_bbox[2]>1:
            if yolo_bbox[0] == 0:
                yolo_bbox[2] = 1
            else: 
                yolo_bbox[2] = (crop_width-yolo_bbox[0])/crop_width
        if yolo_bbox[3]>1:
            if yolo_bbox[1] == 0:
                yolo_bbox[3] = 1
            else: 
                yolo_bbox[3] = (crop_height-yolo_bbox[1])/crop_height
        # Write annotations in file
        
        annot = "0 " + " ".join(map(str, yolo_bbox )) + "\n"

        file.write(annot)

In [ ]:
image_list = os.listdir("./images")
print(len(image_list))

image_list = os.listdir("./labels")
print(len(image_list))

In [ ]:
# Training, validation and test data
train_data = df.sample(frac = 0.7, random_state = 2) #[train["video_id"].isin([0, 2])]
test_data = df.drop(train_data.index)
valid_data = test_data.sample(frac = 0.9, random_state = 2)#train[train["video_id"].isin([1])]
test_data = test_data.drop(valid_data.index)
